In [2]:
import anndata as ad
import pickle as pkl
## Download file from : https://figshare.com/ndownloader/files/42706966
test = ad.read_h5ad("../../data/10k_pbmcs_proc.h5ad")

# Get Ensembl IDs for Names

In [3]:
# import requests, sys
# import json
# import pickle as pkl
# server = "https://rest.ensembl.org"
# ext = "/lookup/symbol/homo_sapiens"
# headers={ "Content-Type" : "application/json", "Accept" : "application/json"}

# # r = requests.post(server+ext, headers=headers, data='{ "symbols" : ["BRCA2", "BRAF" ] }')

# gene_id_to_ensemble = dict()

# for i in range(0, len(test.var['gene_symbols']), 1000):
#   print(i+1000,"/",len(test.var['gene_symbols']))
#   symbols = {'symbols':test.var['gene_symbols'][i:i+1000].tolist()}
#   r = requests.post(server+ext, headers=headers, data=json.dumps(symbols))
#   decoded = r.json()
#   gene_id_to_ensemble.update(decoded)
#   # print(repr(decoded))

# pkl.dump(gene_id_to_ensemble, open('./human_gene_to_ensemble_id.pkl', 'wb')) 

In [4]:
mappings = pkl.load(open('./human_gene_to_ensemble_id.pkl', 'rb'))

In [5]:
test.var['ensembl_id'] = test.var['gene_symbols'].apply(lambda x: mappings.get(x,{"id":None})['id'])

In [6]:
test.write_h5ad("./data_genetf/10k_pbmcs_proc_genetf.h5ad")

In [7]:
## Install Geneformer  from https://huggingface.co/ctheodoris/Geneformer
## TODO : Use direct code snippets instead of cloning the full repo

In [8]:
from geneformer import TranscriptomeTokenizer

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.
/opt/homebrew/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
tk = TranscriptomeTokenizer({"cell_type": "cell_type"}, nproc=4)
data_dir = "./data_genetf"
output_dir = "./data_genetf"
output_prefix = "genetransformer"

In [14]:
tk.tokenize_data(data_dir, 
                 output_dir,
                 output_prefix,
                 file_format="h5ad")

Tokenizing data_genetf/10k_pbmcs_proc_genetf.h5ad
data_genetf/10k_pbmcs_proc_genetf.h5ad has no column attribute 'filter_pass'; tokenizing all cells.


/opt/homebrew/lib/python3.11/site-packages/anndata/__init__.py:51: FutureWarning: `anndata.read` is deprecated, use `anndata.read_h5ad` instead. `ad.read` will be removed in mid 2024.
  warnings.warn(


Creating dataset.


Saving the dataset (1/1 shards): 100%|██████████| 11990/11990 [00:00<00:00, 496399.18 examples/s]


In [12]:
from geneformer import EmbExtractor

# initiate EmbExtractor
embex = EmbExtractor(model_type="Pretrained",
                     num_classes=0,
                     filter_data=None,
                     max_ncells=10,
                     emb_layer=0,
                     emb_label=None,
                     labels_to_plot=None,
                     forward_batch_size=5,
                     nproc=4)


In [15]:

# extracts embedding from input data
# input data is tokenized rank value encodings generated by Geneformer tokenizer (see tokenizing_scRNAseq_data.ipynb)
# example dataset: https://huggingface.co/datasets/ctheodoris/Genecorpus-30M/tree/main/example_input_files/cell_classification/disease_classification/human_dcm_hcm_nf.dataset
embs = embex.extract_embs("/Users/maximeallard/helical/git/Geneformer/geneformer-12L-30M",
                          "/Users/maximeallard/helical/git/helical-package/helical/models/geneformer/data_genetf/genetransformer.dataset",
                          "./data_genetf",
                          "results_emb")

100%|██████████| 2/2 [00:06<00:00,  3.09s/it]


In [16]:
embs.shape

(10, 512)